In [1]:
import math
import pandas as pd
import numpy as np
import scanpy as sc
import anndata as ad
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import scipy.sparse as sp
from torch.utils.data import Dataset, DataLoader

In [2]:
torch.manual_seed(42)

## Dataset

In [3]:
pd_celldata = pd.read_csv("proteinOutFileCat_nn.csv", sep=',', header=0)
#pd_celldata = pd.read_csv("rnaSeqOutFileCat_nn.csv", sep=',', header=0)
pd_celldata = pd_celldata.rename({'Cancer Type': 'cancer_type'}, axis=1)

pd_celldata_t = pd_celldata.drop(columns="Composite.Element.REF")
#pd_celldata_t = pd_celldata.drop(columns="Hugo_Symbol")

celldata_t = pd_celldata.to_numpy()
features = list(pd_celldata)

numlist=[]
labels = []
for ind in range(celldata_t.shape[0]):
    x = celldata_t[ind,:]
    numlist.append(x)
    labels.append(x[-1])

numlist = np.array(numlist)
print(pd_celldata_t)

from sklearn.model_selection import train_test_split

X, y = pd_celldata_t.drop(labels='cancer_type', axis=1),pd_celldata_t['cancer_type']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3,
    stratify=y, random_state=0
)

       A1BG     A2M   A2ML1    AAAS  AACS   AAED1   AAGAB    AAK1   AAMDC  \
0    0.7618  0.7481  0.0000  1.3759   0.0  0.0000  0.0000  0.0160  0.0000   
1    1.4195  0.9259  0.0000  0.6716   0.0  0.0000  0.0808  0.1219  0.0000   
2    0.3180  0.0000  0.0000  0.1638   0.0  0.0000  0.4738  0.0433  0.0000   
3    0.0128  0.0000  0.0000  0.3808   0.0  0.3719  0.0000  0.0169  0.0000   
4    0.0000  0.1080  0.0000  0.0000   0.0  0.0000  0.1073  0.0000  0.0000   
..      ...     ...     ...     ...   ...     ...     ...     ...     ...   
100  0.0771  0.0000  0.7431  0.2510   0.0  0.0000  0.0000  0.1511  0.0000   
101  0.0997  0.0000  0.0000  0.0000   0.0  0.0296  0.1863  0.1723  0.0922   
102  0.7527  0.5561  3.3277  0.8935   0.0  0.0000  0.0000  0.0975  0.0000   
103  0.0126  0.0000  0.0000  0.0796   0.0  0.0000  0.0000  0.2538  0.0000   
104  0.9947  0.4070  0.0000  0.1486   0.0  0.0000  0.0000  0.2440  0.0000   

       AAMP  ...  ZSWIM8    ZW10  ZWILCH   ZWINT    ZXDC  ZYG11B     ZYX  \

## Baseline Gene Rankings

In [4]:
from sklearn.ensemble import RandomForestClassifier  # for classification tasks
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score 
from sklearn.metrics import mean_squared_error

def randomForestBaseline(X_train, y_train, X_test, y_test):

    clf = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust hyperparameters like n_estimators
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {accuracy:.2f}')

    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error: {mse:.2f}')
    
    importances = clf.feature_importances_
    feature_names = list(pd_celldata_t.columns)  # Replace with your actual feature names or column labels

    # Create a list of (importance, feature_name) tuples
    feature_importance_tuples = list(zip(importances, feature_names))
    
    check = feature_importance_tuples.copy()

    # Sort the feature importances in descending order
    feature_importance_tuples.sort(reverse=True, key=lambda x: x[0])

    # Print or access the ranked list of features
    ranked_features = []
    for importance, feature_name in feature_importance_tuples:
        ranked_features.append((feature_name, importance))
    
    return ranked_features

ranked_features = randomForestBaseline(X_train, y_train, X_test, y_test)


Accuracy: 0.72
Mean Squared Error: 0.56


## Normalization

In [5]:
X_train_norm1 = X_train.div(X_train.sum(axis=1), axis=0)
X_train_norm2 = X_train.div(X_train.sum(axis=0), axis=1)
X_train_norm2=X_train_norm2.fillna(0)

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor="white")

X_train_a = np.array(X_train_norm1)
adata = ad.AnnData(X_train_a)
#adata.uns["name"] = "rna_seq"
adata.uns["name"] = "prot"

y_train_str = []
for i in y_train:
    y_train_str.append(str(i)) # convert to strings so that they can be recognized by scanpy
adata.obs['true_labels'] = y_train_str


scanpy==1.9.6 anndata==0.10.3 umap==0.5.4 numpy==1.24.3 scipy==1.11.1 pandas==2.0.3 scikit-learn==1.3.0 statsmodels==0.14.0 pynndescent==0.5.10


## preprocessing 

In [6]:
adata.obs

,true_labels
0,1
1,3
2,1
3,3
4,3
...,...
68,3
69,0
70,4
71,4


In [7]:
class MyDataset(Dataset):
    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def __len__(self):
        return len(self.X_train)

    def __getitem__(self, idx):
        X = self.X_train.iloc[idx].values  # Extract values from X_train DataFrame row
        y = self.y_train.iloc[idx]  # Extract label from y_train DataFrame
        return torch.tensor(X, dtype=torch.float), torch.tensor(y, dtype=torch.long)  # Convert to PyTorch tensors

def preprocess(adata):
    sc.pp.filter_genes(adata, min_cells=1)
    sc.pp.normalize_total(adata)
    sc.pp.neighbors(adata, use_rep='X')
    return adata

In [12]:
# Define an Autoencoder (AE) model 

##### Original ####


class AE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(AE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 4*1024),
            nn.ReLU(),
            nn.Linear(4*1024, 4*512),
            nn.ReLU(),
            nn.Linear(4*512, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 4*512),
            nn.ReLU(),
            nn.Linear(4*512, 4*1024),
            nn.ReLU(),
            nn.Linear(4*1024, input_dim),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        latent_representation = self.encoder(x)
        recon_batch = self.decoder(latent_representation)
        return recon_batch, latent_representation


In [13]:


# Define a Multi-Layer Perceptron (MLP) model 
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_prob=0.5):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.dropout = nn.Dropout(dropout_prob)  # Dropout layer
        self.fc3 = nn.Linear(256, output_dim)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)  # Apply dropout
        x = self.fc3(x)
        return x
    

# Combine AE and MLP models
class CombinedModel(nn.Module):
    def __init__(self, ae, mlp):
        super(CombinedModel, self).__init__()
        self.ae = ae
        self.mlp = mlp
    
    def forward(self, x):
        recon_batch, latent_representation = self.ae(x)
        mlp_output = self.mlp(latent_representation)
        return recon_batch, mlp_output




In [16]:
latent_dim = 64
#input_dim = len(adata.var)
input_dim = len(numlist[0]) - 2

output_dim = 5
#reaches 78.12% with output_dim=10 but stabalizes at 75%
batch_size = 757
learning_rate = 0.001

num_epochs = 100

# Initialize AE and MLP models with increased complexity and dropout
ae = AE(input_dim, latent_dim)
mlp = MLP(latent_dim, output_dim)

# Create the combined model
combined_model = CombinedModel(ae, mlp)

autoencoder_criterion = nn.MSELoss()
mlp_criterion = nn.CrossEntropyLoss()

optimizer_combined = optim.AdamW(combined_model.parameters(), lr=0.001)

# Initialize the learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer_combined, step_size=5, gamma=0.5)



In [17]:
# get data loaders

X_df_train = pd.DataFrame(X_train)
y_df_train = pd.Series(y_train)

dataset_train = MyDataset(X_df_train, y_df_train)
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

X_df_test = pd.DataFrame(X_test)
y_df_test = pd.Series(y_test)

dataset_test = MyDataset(X_df_test, y_df_test)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)


In [18]:
def composite_loss(autoencoder_loss, mlp_loss, custom_loss_weight):
    return autoencoder_loss + mlp_loss + (custom_loss_weight * custom_loss)


In [19]:
# Define your custom MSE loss function.
class CustomMSELoss(nn.Module):
    def __init__(self, gene_ranking, gene_importance_threshold, learning_rate):
        super(CustomMSELoss, self).__init__()
        self.gene_ranking = nn.Parameter(gene_ranking.clone().detach(), requires_grad=True)
        self.gene_importance_threshold = gene_importance_threshold
        self.learning_rate = learning_rate

    def forward(self, predicted, target):
        mse_loss = torch.mean((predicted - target)**2)

        # Penalize less important genes
        gene_penalty = torch.sum(torch.relu(self.gene_ranking - self.gene_importance_threshold))
        total_loss = mse_loss + gene_penalty

        return total_loss


In [21]:
gene_importance_threshold = 0.1
#custom_loss = CustomMSELoss(ordered, gene_importance_threshold, learning_rate)

num_epochs = 50
# Training loop
for epoch in range(num_epochs):
    for batch_idx, (X_train, y_train) in enumerate(train_loader):
        X_train = X_train.view(X_train.size(0), -1)

        optimizer_combined.zero_grad()
        #X_train = X_train.view(batch_size, 1, 73, 9733)

        recon_batch, mlp_output_train = combined_model(X_train)

        # Calculate AE loss (reconstruction loss)
        #ae_loss = custom_loss(recon_batch, data)
        
        ae_loss = nn.MSELoss()(recon_batch, X_train)

        # Calculate MLP loss 
        mlp_loss = nn.CrossEntropyLoss()(mlp_output_train, y_train)

        # Total loss
        total_loss = ae_loss + mlp_loss

        optimizer_combined.zero_grad()
        total_loss.backward(retain_graph=True)


        optimizer_combined.step()

        #ordered.grad = torch.autograd.grad(ae_loss, custom_loss.gene_ranking)[0]

        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx}/{len(train_loader)}], AE Loss: {ae_loss.item():.4f}, MLP Loss: {mlp_loss.item():.4f}')

    # Step the learning rate scheduler
    scheduler.step()

    # Evaluate accuracy on validation/test dataset
    combined_model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for X_test, y_test in test_loader:  
            X_test = X_test.view(X_test.size(0), -1)
            _, mlp_output = combined_model(X_test)
            _, predicted = torch.max(mlp_output, 1)
            total += y_test.size(0)
            correct += (predicted == y_test).sum().item()



    accuracy = correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Combined Model Accuracy on Validation/Test dataset: {100 * accuracy:.2f}%')

Epoch [1/50], Batch [0/1], AE Loss: 0.2168, MLP Loss: 1.6082
Epoch [1/50], Combined Model Accuracy on Validation/Test dataset: 31.25%
Epoch [2/50], Batch [0/1], AE Loss: 0.1500, MLP Loss: 1.5045
Epoch [2/50], Combined Model Accuracy on Validation/Test dataset: 28.12%
Epoch [3/50], Batch [0/1], AE Loss: 0.1481, MLP Loss: 2.2803
Epoch [3/50], Combined Model Accuracy on Validation/Test dataset: 40.62%
Epoch [4/50], Batch [0/1], AE Loss: 0.1571, MLP Loss: 1.3515
Epoch [4/50], Combined Model Accuracy on Validation/Test dataset: 50.00%
Epoch [5/50], Batch [0/1], AE Loss: 0.1431, MLP Loss: 1.2311
Epoch [5/50], Combined Model Accuracy on Validation/Test dataset: 53.12%
Epoch [6/50], Batch [0/1], AE Loss: 0.1412, MLP Loss: 1.0072
Epoch [6/50], Combined Model Accuracy on Validation/Test dataset: 53.12%
Epoch [7/50], Batch [0/1], AE Loss: 0.1394, MLP Loss: 0.8908
Epoch [7/50], Combined Model Accuracy on Validation/Test dataset: 53.12%
Epoch [8/50], Batch [0/1], AE Loss: 0.1369, MLP Loss: 0.7853
E

In [62]:

# Convergence criteria
max_iterations = 10  # Adjust as needed
convergence_threshold = 0.001  # Adjust as needed

for iteration in range(max_iterations):
    # Train the model with the current gene rankings
    for epoch in range(num_epochs):
        for batch_idx, (data, labels) in enumerate(train_loader):
            data = data.view(data.size(0), -1)
            # Forward pass, compute loss, and backpropagate
            # Use the custom loss function with the current gene rankings
            optimizer_combined.zero_grad()
            gene_rankings = ordered.clone().detach().requires_grad_(True)
            recon_batch, mlp_output = combined_model(data)
            print(recon_batch)
            print(mlp_output)
            print(labels)
            loss = CustomLoss(ordered)(latent_representation, labels)
            loss.backward()
            optimizer.step()

        # Check for convergence by comparing updated gene rankings
        new_gene_rankings = ordered.detach().numpy()
        if torch.norm(new_gene_rankings - ordered) < convergence_threshold:
            break  # Converged

        # Update gene rankings based on the current model (you can define your update logic)
        ordered = new_gene_rankings

# Final gene rankings after convergence
final_gene_rankings = new_gene_rankings



NameError: name 'ordered' is not defined

In [ ]:
print(ae_loss)
print(mlp_loss)